In [105]:
def parse_fasta_to_df(file_path, content_alias, id_alias = "ProtID"):
    from Bio import SeqIO
    import pandas as pd
    with open(file_path) as fasta_file:
        ids = []
        contents = []
        for record in SeqIO.parse(fasta_file, 'fasta'):
            ids.append(record.id)
            contents.append(record.seq)
        df = pd.DataFrame(data = {id_alias: ids, content_alias: contents})
        return df.set_index(id_alias)
    
def filter_unique_rownames(df):
    return df[~df.index.duplicated(keep='first')]


def import_protvec(filepath, namescol = "words"):
    """
    Import data frame of ProtVec 3-grams. 
    
    :param filepath: path to a TSV.
    :param namescol: name of a column with row names 
    :return: pandas dataframe with 3-grams as rownames.
    """
    import pandas as pd
    protvec_df = pd.read_csv(filepath, sep = "\t", header = 0)
    protvec_df_3gramidx = protvec_df.set_index(namescol)
    return(protvec_df_3gramidx)

def get3gramvec(threegr_df, threegr_name, as_list = False):
    if not threegr_name in threegr_df.index:
        raise ValueError(''.join(["The supplied ProtVec dataset is not trained for the threegram: ", threegr_name]))
    vec = threegr_df.loc[threegr_name].values
    if (as_list):
        vec = vec.tolist()
    return vec
    
def convert_seq_to_protvec(seq, threegr_df, substitute_any_with="G"):
    """
    Get ProtVec representation of a given sequence
    """
    import numpy as np
    protvec = np.zeros(100)
    for i in range(0, len(seq) - 3):
        this3gram = str(seq[i:i+3])
        this3gram = this3gram.replace("X", substitute_any_with)
        if not this3gram in threegr_df.index:
            # skip untrained 3grams
            continue
        this3gramvec = get3gramvec(threegr_df, this3gram)

        protvec = np.add(protvec, this3gramvec)
    # Exploratory DATA Analysis (EDA)
    # add amino acid sequence length
    protvec = np.append(protvec, get_sequence_len(seq))
    
    return protvec

def get_sequence_len(seq):
    count = 0
    for char in seq:
        count+=1
    return count

In [102]:
import pandas as pd
import numpy as np
import os

# get current dir/filename
import inspect
this_file_path = os.path.abspath(inspect.getframeinfo(inspect.currentframe()).filename)
this_dir = os.path.dirname(this_file_path)

class_path = os.path.join(this_dir, "data", "PP_step1_trn.class")
seq_path = os.path.join(this_dir, "data", "PP_step1_trn.fas")
protvec_file = os.path.join(this_dir, "data", "protVec_100d_3grams.csv")

# parse information
class_df = filter_unique_rownames(parse_fasta_to_df(class_path,"Binds"))
seq_df = filter_unique_rownames(parse_fasta_to_df(seq_path,"Sequence"))

# Convert levels to numeric
class_df['Binds'] = class_df['Binds'].map({"Bind" : 1, "Non_Bind" : 0})


# Get Bind/non-bind ratio
counts = class_df['Binds'].value_counts()
ratio = counts[0]/counts[1] # 1: 536, # 0: 369
print("Bind/non-bind ratio is", str(ratio))
# print(class_df['Binds'].value_counts())

Bind/non-bind ratio is 0.688432835821


/Users/universal/anaconda3/lib/python3.6/site-packages/Bio/Seq.py:163: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [106]:
# Convert sequences to their ProtVec representation
pv_df = import_protvec(protvec_file)
seq_df['Sequence'] = seq_df['Sequence'].apply(convert_seq_to_protvec, args = (pv_df,))

# Merge two dataframes into one - REDUNDANT
protvec_and_target_df = pd.merge(class_df, seq_df, left_index=True, right_index=True)


In [107]:
# TODO (?): access individual columns from the df
protvec_columns_df = pd.DataFrame(protvec_and_target_df['Sequence'].values.tolist())
protvec_columns_df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,-4.945520,-1.101556,1.377135,-4.704273,0.024413,-1.343140,1.622015,-0.418979,-0.602140,3.056220,...,-1.120882,0.413605,-0.956793,3.586979,-0.362186,-0.128984,-2.984049,-0.496378,5.143955,72.0
1,-27.965312,-7.714547,2.730592,-25.785597,-5.746298,-3.273256,16.017948,0.228426,-4.485861,20.641059,...,-9.554774,-0.903278,6.867601,23.158106,2.777006,-5.594944,-14.862038,-3.015878,17.770602,386.0
2,-16.271814,-0.640101,5.404184,-8.445870,-2.753056,2.258669,6.453967,1.175087,-0.689992,15.564769,...,-9.586030,-2.974845,2.331137,3.657300,0.629176,-4.017741,-1.269229,-5.090055,9.409096,162.0
3,-7.488620,-3.736660,0.497572,-9.259213,-0.148268,-1.831033,2.428981,1.061687,1.376447,5.911929,...,-4.834465,-2.273275,-0.530321,3.658159,0.816585,-1.756309,-4.778736,-3.369866,9.570885,117.0
4,-8.233843,-2.502467,-4.157566,-10.052415,1.336235,-0.626557,-0.905718,-3.420479,-4.922915,9.705653,...,1.137749,-6.972810,-0.711821,1.653670,4.915066,-3.785126,-9.397865,-8.140359,15.543700,147.0
5,-6.723216,-2.230543,-5.659682,-13.655185,2.334752,-1.769039,0.692337,-6.427726,-7.426591,5.282965,...,1.037608,-5.301468,0.167031,7.269997,3.843122,-1.650813,-14.584181,-2.391833,19.672105,184.0
6,-5.445150,0.415351,2.689022,-6.631331,0.540731,-1.684854,2.939287,-0.765284,-1.416537,3.720832,...,-1.883490,-0.074298,-0.106026,1.245721,1.474673,-2.140444,-2.664437,0.182383,4.785870,84.0
7,-10.562770,-3.956826,0.227718,-9.661443,-4.029101,-0.172810,2.490740,1.193392,3.067679,11.599946,...,-6.357659,-3.017517,3.482338,5.656827,2.023493,-3.051422,-9.407004,-10.620249,13.577846,163.0
8,-14.193158,-4.443981,0.099315,-10.816634,1.079201,-1.164256,4.284271,3.225812,-2.797150,15.049184,...,-2.010290,-2.337543,1.025977,6.379547,-1.176908,-1.038982,-7.849447,-6.691045,8.916508,184.0
9,-9.495451,-2.119338,3.963915,-6.633833,-3.278948,-0.465050,4.591717,0.927544,2.268823,7.642591,...,-5.896460,2.426519,2.500563,6.031022,1.203915,-3.703630,-2.718476,-2.908347,8.800904,122.0


In [108]:
# Merge two dataframes into one
protvec_and_target_df = pd.merge(class_df, seq_df, left_index=True, right_index=True)

In [109]:
# Eject features and targets, make CV splits
from sklearn.model_selection import StratifiedKFold
N_SPLITS_CV = 5

#features = np.stack(protvec_and_target_df['Sequence'].values.tolist(), axis = 0)
features = np.stack(protvec_and_target_df['Sequence'].values.tolist(), axis = 0)
targets = protvec_and_target_df['Binds'].values # == labels

skf = StratifiedKFold(n_splits=N_SPLITS_CV)
folds = skf.split(features, targets)

for train_index, test_index in folds:
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = targets[train_index], targets[test_index]# test layer

In [110]:
##### Fit ANN # http://scikit-learn.org/stable/modules/neural_networks_supervised.html
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

X = X_train
y = y_train


clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(6, 279), random_state=1)
clf.fit(X, y)    

# Predict sample
clf.predict(X_test)

scores = cross_val_score(clf, features, targets, cv = skf)
print(scores)
score_sum = 0
for score in scores:
    score_sum += score
score_avg = score_sum / len(scores)
print(score_avg)

[ 0.64835165  0.63535912  0.64640884  0.64640884  0.67777778]
0.650861244342


**Sci-kit GridSearchCV**

https://datascience.stackexchange.com/questions/36049/how-to-adjust-the-hyperparameters-of-mlp-classifier-to-get-more-perfect-performa

https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search


In [111]:
print("Training features shape: ", X_train.shape)
print("Testing features shape: ", X_test.shape)

Training features shape:  (725, 101)
Testing features shape:  (180, 101)


In [112]:
import numpy as np


# hidden layers: generates list of n_max tuples with 
# length: n_l_min--n_l_max integers, value: each between n_a_min and n_a_max
# https://www.kaggle.com/jilkoval/titanic-with-random-forest-and-neural-networks
def rand_hidden_layer_sizes(n_l_min,n_l_max,n_a_min,n_a_max,n_max=1000):
    n_l = np.random.randint(n_l_min,n_l_max,n_max)
    list_hl = []
    for nl_i in n_l:
        list_hl.append(tuple(np.random.randint(n_a_min,n_a_max,nl_i)))
    return list_hl

hidden_layers = rand_hidden_layer_sizes(1,3,1,10000,500)

[(8962, 928), (7494, 4382), (1291, 5241), (3735,), (1881, 9557), (7240, 3518), (4857,), (9612,), (6154,), (2596, 408), (9692,), (884,), (9853, 7904), (2419, 6920), (3448,), (1045,), (3266,), (6205, 7043), (7951,), (6296, 538), (8510,), (2639, 5687), (534,), (4549,), (5714,), (1983, 2772), (3300, 8957), (381,), (6659, 7736), (6298, 746), (5465,), (1254, 4892), (4062, 5946), (1990,), (8613,), (173,), (9078,), (8619,), (1431, 4997), (9872, 6325), (3170,), (9966, 4111), (9396,), (900,), (4811,), (7044, 6758), (9948, 9105), (9542,), (2140,), (2360,), (9314, 7823), (7559, 3245), (8772, 3223), (5860,), (7448, 5663), (9749,), (5739,), (2849,), (2034, 2020), (1741,), (6888, 5219), (6282,), (9235,), (1927,), (5456,), (8065, 8066), (2126, 6523), (2976,), (6213, 6797), (2938, 4224), (9420, 2138), (4545,), (2433, 5701), (3312,), (7982,), (7967,), (9159,), (5541,), (9371,), (4354, 6203), (7625, 6068), (931,), (7434,), (3337, 6590), (2842,), (8219,), (5070, 8023), (4337, 5324), (4551, 6803), (4392,),

In [ ]:
# defining parameter space to search
# solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(6, 279), random_state=1
parameter_space = {
'hidden_layer_sizes': list(range(5000, 30000)) # https://stackoverflow.com/a/52029734
,'activation' : ['relu', 'identity', 'logistic', 'tanh'] #'
,'solver': ['lbfgs'] # lbfgs performs well on small datasets
,'alpha': list(np.linspace(0.00001, 1, 100))
,'batch_size' : ['auto'] # not used when solver is lbfgs
,'learning_rate' : ['constant','invscaling','adaptive'] # only with solver l
,'learning_rate_init' : list(np.linspace(0.001, 1, 100))
,'max_iter' : [50, 100, 150, 300, 600]
,'random_state' : [6] # sets the seed
,'warm_start' : [False, True]
,'tol' : [0.0001, 0.001, 0.01, 0.1, 1]
,'verbose' : [True] # prints progress to stdout
}




# parameters not used
# ,'power_t' : [0.5, ] # only when solver is SGD
# #'early_stopping' : [False], # only with SGD or ADAM
#'beta_1' : [0.9], # not with LBFGS
#'beta_2' : [0.999], # not with LBFGS
#'epsilon' : [1e-8] # not with LBFGS
#,'n_iter_no_change' : [10] # not with LBFGS
#'nesterovs_momentum' : [True], # only with SGD
#'momentum' : [0.9], # only with SGD
#'shuffle' : [True], # only with SGD or ADAM
#'validation_fraction' : [0.1] # not with LBFGS

In [ ]:
# first randomized search to find roughly the best attributes
# choosing classifier
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=150) # max_iter will be constant

# running the search
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(mlp, parameter_space, cv=3, scoring='accuracy')
random_search.fit(X, y)


# Best paramete set
print('Best parameters found:\n', random_search.best_params_)

# All results
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    
best_model = random_search.best_estimator_

In [ ]:
# best scores so far with RandomizedSearchCV:
# 0.687 (+/-0.024) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 50, 'learning_rate_init': 0.80827272727272736, 'learning_rate': 'adaptive', 'hidden_layer_sizes': 8585, 'batch_size': 'auto', 'alpha': 0.19192727272727272, 'activation': 'logistic'}
# 0.683 (+/-0.028) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 50, 'learning_rate_init': 0.8183636363636364, 'learning_rate': 'constant', 'hidden_layer_sizes': 9900, 'batch_size': 'auto', 'alpha': 0.92929363636363627, 'activation': 'logistic'}
# 0.672 (+/-0.031) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 150, 'learning_rate_init': 0.17254545454545456, 'learning_rate': 'invscaling', 'hidden_layer_sizes': 7369, 'batch_size': 'auto', 'alpha': 0.29293636363636366, 'activation': 'logistic'}
# 0.665 (+/-0.034) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 50, 'learning_rate_init': 0.27345454545454545, 'learning_rate': 'constant', 'hidden_layer_sizes': 1800, 'batch_size': 'auto', 'alpha': 0.001, 'activation': 'logistic'}
# 0.663 (+/-0.024) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 150, 'learning_rate_init': 0.63672727272727281, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4115,), 'batch_size': 'auto', 'alpha': 1e-05, 'activation': 'logistic'}
# 0.652 (+/-0.090) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 100, 'learning_rate_init': 0.16245454545454546, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (17, 16, 14, 5, 19, 14, 15, 13, 14, 9, 11, 11), 'batch_size': 'auto', 'alpha': 1e-05, 'activation': 'identity'}
# 0.650 (+/-0.036) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 50, 'learning_rate_init': 0.33400000000000002, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (97, 14, 30, 59), 'batch_size': 'auto', 'alpha': 0.0001, 'activation': 'identity'}
# 0.650 (+/-0.025) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 150, 'learning_rate_init': 0.62663636363636366, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (27, 58, 30), 'batch_size': 'auto', 'alpha': 0.0001, 'activation': 'relu'}
# 0.646 (+/-0.022) for {'verbose': True, 'solver': 'lbfgs', 'random_state': 4, 'power_t': 0.5, 'max_iter': 50, 'learning_rate_init': 0.7477272727272728, 'learning_rate': 'adaptive', 'hidden_layer_sizes': 700, 'batch_size': 'auto', 'alpha': 0.001, 'activation': 'relu'}

# testing the best parameter
best_parameter = {'verbose': True, 'solver': 'lbfgs', 'random_state': 6, 'power_t': 0.5, 'max_iter': 50, 'learning_rate_init': 0.27345454545454545, 'learning_rate': 'constant', 'hidden_layer_sizes': 1800, 'batch_size': 'auto', 'alpha': 0.001, 'activation': 'logistic'}

X = X_train
y = y_train


# TODO: Find a way to evaluate a dictionary


clf = MLPClassifier(verbose=True, solver = 'lbfgs', random_state=6, power_t=0.5, max_iter=50, learning_rate_init=0.27345454545454545, learning_rate='constant', hidden_layer_sizes= 1800, batch_size= 'auto', alpha = 0.001, activation = 'logistic')
clf.fit(X, y)    

# Predict sample
clf.predict(X_test)

scores = cross_val_score(clf, features, targets, cv = skf)
print(scores)

In [ ]:
# then performing GridSearchCV to get even better results
# choosing classifier
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=3)

# running the search
from sklearn.model_selection import GridSearchCV

random_search = GridSearchCV(mlp, parameter_space, cv=3, scoring='accuracy')
random_search.fit(X, y)

# 
# Best paramete set
print('Best parameters found:\n', random_search.best_params_)

# All results
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))